# Real-Time Experimental Results Visualizer

## Setup

In [ ]:
import plotly.graph_objects as go
import pymem.utils.pipeline

In [ ]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

In [ ]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

In [ ]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

## Place your own settings here

In [ ]:
# Input and output paths are relative to root directory
INTEGER_ARG = 1
OUTPUT_PATH = os.path.join("TEST")
SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
CONFIG_ARGS = pymem.utils.pipeline.ConfigArgs(
    path=os.path.join("TEST", "dataset")
    , inst="CompleteBipartiteSum"
    , seed=42
    , xlsx=False
    , step=0
    , rows=100
    , cols=1
    , itrs=1
    , pool=1
)

## Run the following cell for real-time results

In [ ]:
# Column to plot must be one of:
# 'peak_nonpaged_pool', 'peak_paged_pool', 'peak_pagefile', 'nonpaged_pool', 
# 'paged_pool', 'peak_wset', 'pagefile', 'private', 'wset', 'rss', 'uss', 'vms'
pymem.utils.pipeline.run_vis(CONFIG_ARGS
  , OUTPUT_PATH
  , INTEGER_ARG
  , SOFFICE_DIR
  , fv_fig
  , vo_fig
  , barplt
  , 'wset'
)